In [52]:
import pyarrow
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

        16
        17
        18
        19

In [2]:
path = 'data/yellow_tripdata_2022-01.parquet'

In [3]:
df = pd.read_parquet(path)

In [4]:
len(df)

2463931

In [5]:
len(df.columns)

19

Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

        41.45
        46.45
        51.45
        56.45

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [7]:
df.isnull().sum()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71503
trip_distance                0
RatecodeID               71503
store_and_fwd_flag       71503
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71503
airport_fee              71503
dtype: int64

In [8]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].apply(lambda x: x.total_seconds()/60 )
df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'duration']]

,tpep_pickup_datetime,tpep_dropoff_datetime,duration
0,2022-01-01 00:35:40,2022-01-01 00:53:29,17.816667
1,2022-01-01 00:33:43,2022-01-01 00:42:07,8.400000
2,2022-01-01 00:53:21,2022-01-01 01:02:19,8.966667
3,2022-01-01 00:25:21,2022-01-01 00:35:23,10.033333
4,2022-01-01 00:36:48,2022-01-01 01:14:20,37.533333
...,...,...,...
2463926,2022-01-31 23:36:53,2022-01-31 23:42:51,5.966667
2463927,2022-01-31 23:44:22,2022-01-31 23:55:01,10.650000
2463928,2022-01-31 23:39:00,2022-01-31 23:50:00,11.000000
2463929,2022-01-31 23:36:42,2022-01-31 23:48:45,12.050000


In [9]:
df['duration'].describe()

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64

In [10]:
df['duration'].describe()['std']

46.44530513776499

Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

        90%
        92%
        95%
        98%

In [11]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [12]:
df_new = df.loc[(df['duration'] >= 1) & (df['duration'] <= 60)]
df_new

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN,5.966667
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN,10.650000
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN,11.000000
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN,12.050000


In [13]:
len(df_new)/ len(df)*100

98.27547930522405

Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

        2
        155
        345
        515
        715

In [14]:
my_df = df_new[['PULocationID', 'DOLocationID', 'duration']]
#my_df.dtypes
my_df

,PULocationID,DOLocationID,duration
0,142,236,17.816667
1,236,42,8.400000
2,166,166,8.966667
3,114,68,10.033333
4,68,163,37.533333
...,...,...,...
2463926,90,170,5.966667
2463927,107,75,10.650000
2463928,113,246,11.000000
2463929,148,164,12.050000


In [15]:
my_df.dtypes

PULocationID      int64
DOLocationID      int64
duration        float64
dtype: object

In [35]:
categorical_col = ['PULocationID', 'DOLocationID']
my_df[categorical_col] = my_df[categorical_col].astype(str)
my_df.dtypes

/tmp/ipykernel_5266/1682616757.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df[categorical_col] = my_df[categorical_col].astype(str)


PULocationID     object
DOLocationID     object
duration        float64
dtype: object

In [17]:
dv = DictVectorizer()

In [37]:
my_df_dict = my_df[categorical_col].to_dict(orient = 'records')

In [38]:
my_df_dict

[{'PULocationID': '142', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '42'},
 {'PULocationID': '166', 'DOLocationID': '166'},
 {'PULocationID': '114', 'DOLocationID': '68'},
 {'PULocationID': '68', 'DOLocationID': '163'},
 {'PULocationID': '138', 'DOLocationID': '161'},
 {'PULocationID': '233', 'DOLocationID': '87'},
 {'PULocationID': '238', 'DOLocationID': '152'},
 {'PULocationID': '166', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '141'},
 {'PULocationID': '141', 'DOLocationID': '229'},
 {'PULocationID': '114', 'DOLocationID': '90'},
 {'PULocationID': '234', 'DOLocationID': '113'},
 {'PULocationID': '246', 'DOLocationID': '79'},
 {'PULocationID': '43', 'DOLocationID': '140'},
 {'PULocationID': '239', 'DOLocationID': '151'},
 {'PULocationID': '148', 'DOLocationID': '141'},
 {'PULocationID': '237', 'DOLocationID': '107'},
 {'PULocationID': '7', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '263'},
 {'PULocationID': '263', 'DOLoc

In [39]:
X_train = dv.fit_transform(my_df_dict)

In [40]:
X_train

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [41]:
X_train.shape

(2421440, 515)

Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

        6.99
        11.99
        16.99
        21.99

In [49]:
y_train = my_df['duration'].values

In [50]:
y_train

array([17.81666667,  8.4       ,  8.96666667, ..., 11.        ,
       12.05      , 27.        ])

In [51]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [55]:
y_pred = lr.predict(X_train)

In [56]:
rmse = mean_squared_error(y_train, y_pred, squared=False)
print(f" the rmse on the training data is {rmse}")

 the rmse on the training data is 6.986190835307031


Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

        7.79
        12.79
        17.79
        22.79

In [63]:
def read_dataframe(file_name):
    df = pd.read_parquet(file_name)

    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda x: x.total_seconds()/60)
    df = df[((df.duration >= 1) & (df.duration <= 60))]

    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] =  df[categorical].astype(str)

    return df    

In [64]:
df_eval = read_dataframe('data/yellow_tripdata_2022-02.parquet')

In [67]:
df_eval 

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.00,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.00,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.50,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.00,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.50,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979426,2,2022-02-28 23:50:00,2022-03-01 00:06:00,NaN,3.40,NaN,None,163,193,0,14.37,0.00,0.5,0.00,0.00,0.3,17.67,NaN,NaN,16.000000
2979427,2,2022-02-28 23:06:57,2022-02-28 23:19:12,NaN,3.48,NaN,None,141,4,0,14.51,0.00,0.5,2.00,0.00,0.3,19.81,NaN,NaN,12.250000
2979428,2,2022-02-28 23:48:13,2022-03-01 00:03:33,NaN,3.05,NaN,None,161,151,0,14.38,0.00,0.5,3.81,0.00,0.3,21.49,NaN,NaN,15.333333
2979429,2,2022-02-28 23:56:41,2022-03-01 00:04:57,NaN,2.62,NaN,None,141,226,0,12.53,0.00,0.5,1.71,0.00,0.3,17.54,NaN,NaN,8.266667


In [70]:
df_eval[categorical_col].to_dict(orient='records')

[{'PULocationID': '138', 'DOLocationID': '252'},
 {'PULocationID': '138', 'DOLocationID': '41'},
 {'PULocationID': '138', 'DOLocationID': '200'},
 {'PULocationID': '239', 'DOLocationID': '200'},
 {'PULocationID': '138', 'DOLocationID': '125'},
 {'PULocationID': '140', 'DOLocationID': '142'},
 {'PULocationID': '140', 'DOLocationID': '36'},
 {'PULocationID': '48', 'DOLocationID': '68'},
 {'PULocationID': '142', 'DOLocationID': '244'},
 {'PULocationID': '238', 'DOLocationID': '116'},
 {'PULocationID': '132', 'DOLocationID': '36'},
 {'PULocationID': '143', 'DOLocationID': '163'},
 {'PULocationID': '132', 'DOLocationID': '80'},
 {'PULocationID': '234', 'DOLocationID': '225'},
 {'PULocationID': '138', 'DOLocationID': '132'},
 {'PULocationID': '233', 'DOLocationID': '164'},
 {'PULocationID': '186', 'DOLocationID': '262'},
 {'PULocationID': '138', 'DOLocationID': '170'},
 {'PULocationID': '79', 'DOLocationID': '164'},
 {'PULocationID': '132', 'DOLocationID': '225'},
 {'PULocationID': '233', 'D

In [71]:
X_val = dv.transform(df_eval[categorical_col].to_dict(orient='records'))

In [72]:
y_val = df_eval['duration'].values

In [76]:
y_pred = lr.predict(X_val)

In [78]:
rmse_val = mean_squared_error(y_val, y_pred, squared=False)
print(f"THe rmse on the validation dataset is {rmse_val}")

THe rmse on the validation dataset is 7.786407752409839
